# **_Gibbs Sampling for Bayesian Networks_**

_This notebook is used to motivate and explain in detail the code implemented. For further informations check `Davide_Sonno_Gibbs_sampling.pdf` ([file](Davide_Sonno_Gibbs_sampling.pdf))_.

_The whole code is also available as module (*MarkovChainMonteCarlo*) and it is used to produce examples (check_ `MarkovChainMonteCarlo.examples`).

## **Motivations**  
Given the non polynomial nature of the exact inference, the complexity can rapidly increase with the size of the bayesan networks.  
To be able to approximate the true distribution we can use Monte Carlo methods.

### Markov Chains

The key property for this approximation comes from the Markov chains, a random process generating a sequence of states from an initial states, in wich the next states only depends on the last state reached.  

Whenever the algorithm used generates a chain with certain properties, such as visiting all the possible states without getting trapped in loops, we are sure that the amount of time spent in each state corresponds to the probability of that state.  

The other property is called _detailed balance_ and expresses that the flow of probabilities between the states are balanced. If this condition is satisfied that Markov chain can be used to sample the desired probability.

# **Python implementation**

The following sections will show and motivate the code used.

## **Approximate Inference**

This class is a converging point for all the classes of algorithms that compute approximate inference. As for today the only method implemented is **Gibbs Sampling**.

### Gibbs sampling

**_GibbsSampling_** object:
    instantiates `self` using a BayesianNetwork and stores a copy of the *net* and of the CPDs for all the variables. The possible Markov Blankets are also pre-computed given that the net is static.

In [1]:
class GibbsSampling:
    def __init__(self, model):
        from pgmpy.models import BayesianNetwork
        if not isinstance(model, BayesianNetwork):
            raise ValueError("The model is not a pgmpy Bayesian Network")
        model.check_model()
        self.model = model
        self.cpds = model.cpds
        self.variable_values = {
            cpd.variable: cpd.state_names[cpd.variable] for cpd in self.cpds}
        self.blankets = {
            var: self.model.get_markov_blanket(var) 
            for var in self.variable_values.keys()
            }
    
    # def query([...]): ...
    # def check_evidence([...]): ...
    # def check_variables([...]): ...
        

### The *query* method

`query`:
    Execute the approximate inference using the Gibbs Algorithm.

_Parameters_:
- variables: *list*
    variables for which you want to compute the probability
- evidence: *dict*
    (key, value) pair as {var: state_of_var_observed}
    None if no evidence
- N: *int*
    number of samples to be drawn
- rounding: *int*
    number of digits to print
- the remaining variables are unused; they are there for portability with `pgmpy` `query` methods

The steps performed by the algorithm are, in detail:
1) check the values of `variables` and `evidence`
2) ensure that the order of the values in `variables` is the same as in `self.variable_values.keys()`
3) initiate the starting state, using the evidence and randomly drawing the remaining values
4) choose the order in wich draw the variables
5) initiate a `StateCounter` object, holding the counts of the state for the  variables in the query
6) for the number of iterations fixed, execute:
    - take the next variable from the ordering
    - compute it's probability given all the other variables (we only need variables in it's Markov blanket)
    - sample from a uniform variable in [0,1]
    - update the state counter with the new value of the variable
7) return a `QueryResult` object, that will normalize and provide an easy print to read values

In [2]:
class GibbsSampling:
    def query(
            self,
            variables,
            evidence=None,
            virtual_evidence=None,          # unused
            elimination_order="greedy",     #
            joint=True,                     #
            show_progress=True,             #
            N=100_000,
            rounding=4
    ):
        from pgmpy.inference import VariableElimination
        from inference.State import StateCounter
        from query import QueryResult
        from math import ceil
        from random import uniform,choice
        self.check_variables(variables)
        if evidence is not None:
            self.check_evidence(evidence)
        else:
            evidence = dict()
        if N < 1:
            raise ValueError(
                "Number of iterations has to be greater than zero")

        arrange_variables(self.variable_values, variables)

        current_state = self.variable_values.copy()
        for var in current_state.keys():
            if var in evidence.keys():
                current_state[var] = evidence[var]
            else:
                current_state[var] = choice(self.variable_values[var])

        ordering = [var for var in self.variable_values.keys()
                    if var not in evidence.keys()]

        state_counter = StateCounter(self.variable_values, variables)

        for _ in range(ceil(N/len(ordering))):
            for var in ordering:
                current_evidence = {ce: current_state[ce] for ce in self.blankets[var]}
                query = VariableElimination(self.model).query(
                    [var], current_evidence).values
                next_value_index = read_distribution(query, uniform(0, 1))
                current_state[var] = self.variable_values[var][next_value_index]
                state_counter.update(current_state)

        return QueryResult(state_counter, rounding)

    def check_evidence(self, evidence):
        for e in list(evidence.keys()):
            if evidence[e] not in self.variable_values[e]:
                raise ValueError(
                    f"Invalid state for {e}; possible states: {self.variable_values[e]}")

    def check_variables(self, variables):
        for var in variables:
            if var not in list(self.variable_values.keys()):
                raise ValueError(
                    f"The variable {var} is not part of the model")


### Some additional methods

`read_distribution`:
    Returns the index value associated with the given probability

_Parameters_:
- query: *numpy array*
    nparray containing the distribution of a query
- probability: *float*

In [3]:
def read_distribution(query, probability:float):
    step = 0
    for i in range(len(query)):
        if probability <= query[i]+step:
            return i
        step += query[i]
    raise Exception("Next state not found or probability greater than 1")

`arrange_variables`:
    Re-arranges `variables` in-place according to the order of the keys of the dict

_Parameters_:
- values: *dict*
    dict containing the variables and their values
- variables: *iterable*
    variables to arrange accordingly to the dict keys

In [4]:
def arrange_variables(values: dict, variables:iter):
    aux = []
    for v in list(values.keys()):
        if v in variables:
            aux.append(v)
    variables.clear()
    variables.extend(aux)

## **State Counter**

**_StateCounter_** object:
    structure holding the dict of states with the respective counts. It also stores the possible value for each variable and the variables requested in the query. The keys of the dict are tuples containing the values of the variable of the state.

**Methods:**

`update`:
    Updates the count of the states, given a new state

_Parameters_:
- next_state: *dict*
    dict containing values for the variables

In [5]:
from itertools import product


class StateCounter:
    def __init__(self, values: dict, variables: list):
        self.variables = variables
        self.values = values

        aux = [values[var] for var in variables]
        combinations = list(product(*aux))
        self.counter = {}
        for i in range(len(combinations)):
            self.counter[combinations[i]] = 0

    def update(self, next_state):
        values = tuple(next_state[var] for var in self.variables)
        self.counter[values] += 1

    def __str__(self):
        return str(self.counter)

## **Query Result**

**_QueryResult_** object:
    structure holding the normalized values of the counts and provides an easy to read way to print the resulting probability
    
_Parameters:_
- counts: *StateCounter*
    the object used to store the counts of the states
- rounding: *int* 
    optional parameter to be able to print smaller probabilities

In [6]:
class QueryResult:
    def __init__(self, counts: StateCounter,rounding:int):
        self.variables = counts.variables
        self.values = counts.values
        self.counts = counts.counter

        tot=sum(list(self.counts.values()))
        for state in self.counts.keys():
            self.counts[state]/=tot
        self.rounding=rounding

    def __str__(self):
        # lets see how "large" the words to print will be
        max_widths={}
        for var in self.variables:
            max_widths[var]=max(len(f"{var}({v})") for v in self.values[var])
        vars_text=','.join(str(var) for var in self.variables)
        query_text=f'phi({vars_text})'
        max_widths['probability']=len(query_text)+2

        # create the separator for the rows
        row_line=''
        for var in self.variables:
            row_line+=f"+{'-'*(max_widths[var]+2)}"
        row_line+=f"+{'-'*(max_widths['probability']+2)}+\n"

        # create the table header
        header=''
        for var in self.variables:
            header+=f"| {var:<{max_widths[var]}} "
        header+=f"| {query_text:>{max_widths['probability']}} |\n"

        # create the separator between the header and the probabilities
        thick_row_line=''
        for var in self.variables:
            thick_row_line+=f"+{'='*(max_widths[var]+2)}"
        thick_row_line+=f"+{'='*(max_widths['probability']+2)}+\n"

        table=''
        for state in list(self.counts.keys()):
            # create the states
            values = [f"{var}({probability})".ljust(max_widths[var]) for var, probability in zip(self.variables, state)]
            formatted_values = ' | '.join(values)
            formatted_counts = f"{self.counts[state]:.{self.rounding}f}" 
            # create the row
            formatted_row = f"| {formatted_values} | {' ' * (max_widths['probability'] - len(formatted_counts))}{formatted_counts} |"
            table+=formatted_row+'\n'+row_line

        return row_line+header+thick_row_line+table

Example output:

In [7]:
# Gibbs Sampling:
# +-------------+---------------+----------------------------+
# | Burglary    | Earthquake    |   phi(Burglary,Earthquake) |
# +=============+===============+============================+
# | Burglary(0) | Earthquake(0) |                     0.0000 |
# +-------------+---------------+----------------------------+
# | Burglary(0) | Earthquake(1) |                     0.0001 |
# +-------------+---------------+----------------------------+
# | Burglary(1) | Earthquake(0) |                     0.0014 |
# +-------------+---------------+----------------------------+
# | Burglary(1) | Earthquake(1) |                     0.9985 |
# +-------------+---------------+----------------------------+